<a href="https://colab.research.google.com/github/Pakapakk/database-design-for-data-engineering/blob/main/cake_shop_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl ipecho.net/plain
! python -m pip install "pymongo[srv]"==3.11
! pip  install pymongo[srv,tls]
! pip install dnspython
!pip install pandas

In [5]:
# @title
from pymongo.mongo_client import MongoClient
from google.colab import userdata

password = userdata.get('mongoDB_password')

uri = "mongodb+srv://PakkyNoi:" + password + "@workshop-bakery.dltqwqx.mongodb.net/?retryWrites=true&w=majority&appName=Workshop-Bakery"
# Create a new client and connect to the server
client = MongoClient(uri)
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


Pinged your deployment. You successfully connected to MongoDB!


In [6]:
import pandas as pd
from bson import ObjectId

In [7]:
BakeryDB = client.Bakery

CakeCollection = BakeryDB.cakes
CustomerCollection = BakeryDB.customer_details
OrderCollection = BakeryDB.order_records

In [8]:
# get full cake lists

# allCakeList = [data for data in CakeCollection.find()]
allCakeList = list(CakeCollection.find())

allCakeDF = pd.DataFrame(allCakeList)

allCakeDF

,_id,name,shortDescription,description,image,ingredients,recipe,stock
0,64759d513beb40594cb859e9,Chocolate Cake,Chocolate cake is a cake flavored with melted ...,Chocolate cake is made with chocolate; it can ...,https://addapinch.com/wp-content/uploads/2020/...,"[flour, sugar, cocoa powder]",Preheat your oven to the specified temperature...,25
1,64759e4c3beb40594cb859ed,Cheese Cake,Cheesecake is a sweet dessert consisting of on...,Cheesecake is a sweet dessert consisting of on...,https://sallysbakingaddiction.com/wp-content/u...,"[graham cracker crumbs, sugar, eggs, butter, s...",Mix graham cracker crumbs and melted butter fo...,40
2,664758fe06cdc709b70778de,example,example,example,https://url_example.jpeg,"[A, B, C, D]",example,100


In [9]:
allCustomerList = list(CustomerCollection.find())

allCustomerDF = pd.DataFrame(allCustomerList)

allCustomerDF

,_id,firstname,lastname,age,gender,telephone,email,street,villageNumber,city,postal-code
0,664627cdf94f8b7a6dc89b5e,example,example,18,Male,090,example@gmail.com,example,22/2,example,12345
1,664761bd06cdc709b70778df,Veggiee,Pak,18,Male,090,example@gmail.com,example,22/2,example,12345


In [10]:
#get order details
allOrderList = list(OrderCollection.find())

allOrderDF = pd.DataFrame(allOrderList)

# allOrderList
allOrderDF

,_id,firstname,lastname,vegan,menu,quantity,orderingDate,pickupDate,pickupLocation
0,66486239b58df0a4120a2075,Veggiee,Pak,No,Cheese Cake,0,2024-05-18,2024-05-31,Self-pickup
1,66486251b58df0a4120a2076,example,example,No,Chocolate Cake,0,2024-05-18,2024-05-31,Self-pickup
2,664865c6b58df0a4120a2077,Veggiee,Pak,No,Chocolate Cake,0,2024-05-18,2024-06-08,Delivery


In [26]:
#get customer details from the order

def getCustomerInfoByOrderID(order_id, OrderCollection, CustomerCollection):
    orderDocument = OrderCollection.find_one({'_id': ObjectId(order_id)})

    if not orderDocument:
        return pd.DataFrame()

    firstname = orderDocument.get('firstname')
    lastname = orderDocument.get('lastname')

    selectedCustomerInfo = list(CustomerCollection.find({
        'firstname': firstname,
        'lastname': lastname
    }))

    selectedCustomerInfo = pd.DataFrame(selectedCustomerInfo)

    return selectedCustomerInfo

In [27]:
wanted_order_id = "66486239b58df0a4120a2075" #@param {type : "string"}

selectedCustomerInfo = getCustomerInfoByOrderID(wanted_order_id, OrderCollection, CustomerCollection)

selectedCustomerInfo

,_id,firstname,lastname,age,gender,telephone,email,street,villageNumber,city,postal-code
0,664761bd06cdc709b70778df,Veggiee,Pak,18,Male,090,example@gmail.com,example,22/2,example,12345


In [21]:
# make baking plan

print("baking schedule based on pickup date")
bakingSchedule = pd.DataFrame(list(OrderCollection.aggregate([{"$sort":{"pickupDate": 1}}])))

bakingSchedule

baking schedule based on pickup date


,_id,firstname,lastname,vegan,menu,quantity,orderingDate,pickupDate,pickupLocation
0,66486239b58df0a4120a2075,Veggiee,Pak,No,Cheese Cake,0,2024-05-18,2024-05-31,Self-pickup
1,66486251b58df0a4120a2076,example,example,No,Chocolate Cake,0,2024-05-18,2024-05-31,Self-pickup
2,664865c6b58df0a4120a2077,Veggiee,Pak,No,Chocolate Cake,0,2024-05-18,2024-06-08,Delivery


In [28]:
# Customer Order History

def getOrderHistoryByCustomerID(order_id, CustomerCollection, OrderCollection):
    customerDocument = CustomerCollection.find_one({'_id': ObjectId(order_id)})

    if not customerDocument:
        return pd.DataFrame()

    firstname = customerDocument.get('firstname')
    lastname = customerDocument.get('lastname')

    customerOrderHistory = list(OrderCollection.find({
        'firstname': firstname,
        'lastname': lastname
    }))

    customerOrderHistory = pd.DataFrame(customerOrderHistory)

    return customerOrderHistory

In [29]:
wanted_customer_id = "664761bd06cdc709b70778df" #@param {type : "string"}

selectedCustomerHistory = getOrderHistoryByCustomerID(wanted_customer_id, CustomerCollection, OrderCollection)

selectedCustomerHistory

,_id,firstname,lastname,vegan,menu,quantity,orderingDate,pickupDate,pickupLocation
0,66486239b58df0a4120a2075,Veggiee,Pak,No,Cheese Cake,0,2024-05-18,2024-05-31,Self-pickup
1,664865c6b58df0a4120a2077,Veggiee,Pak,No,Chocolate Cake,0,2024-05-18,2024-06-08,Delivery
